## 评估设置

- 设置模型及其路径：在`.env`文件中设置`LLM_MODEL={model-name}` 和 `LLM_MODEL_PATH={model-path}`，其中`LLM_MODEL`为模型名称，`LLM_MODEL_PATH`为模型路径。
- 执行指令：`python dbgpt/app/dbgpt_server.py`，启动服务。
- 接口汇总地址：`http://{your-ip}:5670/docs`

运行该文件时，只需要更改`URL`和`MODEL_NAME`、`DB_FILE` （db文件）即可。



In [15]:
# DB-GPT开放的chat_with_db_execute接口
URL = 'http://172.30.81.108:5670/api/v1/chat/completions'
MODEL_NAME = 'deepseek_proxyllm'
# MODEL_NAME = "qwen2.5-coder-7b-instruct"
DB_FILE = '/root/qjw/projects/DB-GPT/datas/SQLite_test_stage1/qaf.db'

In [16]:
import requests
import re

def send_post_request(url:str, model_name:str, user_input:str, db_file:str):
    """
    发送post请求, 获取数据模型的回复
    """
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json'
    }
    data = {
        "conv_uid": "None",
        "user_input": user_input,
        "user_name": "string",
        "chat_mode": "chat_with_db_execute",
        "select_param": db_file.split("/")[-1].split(".")[0],
        "model_name": model_name,
        "incremental": False,
        "sys_code": "None"
    }
    # 请求接口且时间限制为30s
    response = requests.post(url, headers=headers, json=data, timeout=30)
    return response.content.decode('utf-8')

def parse_data_model_msg(data_model_msg):
    """
    解析数据模型的回复, 提取thoughts, sql, data, type
    """
    # 提取thoughts
    thoughts_content = data_model_msg.split("\n")[0]

    # 提取 sql
    sql_match = re.search(r"&quot;sql&quot;: &quot;(.*?)&quot;, &quot;", data_model_msg)
    sql_data = sql_match.group(1) if sql_match else None
    sql_data = sql_data.replace("&gt;", ">").replace("&lt;", "<")

    # 提取 data
    data_match = re.search(r"&quot;data&quot;: (\[.*?\])", data_model_msg)
    data_data = data_match.group(1) if data_match else None

    # 提取 type
    type_match = re.search(r"&quot;type&quot;: &quot;(.*?)&quot;", data_model_msg)
    type_data = type_match.group(1) if type_match else None

    # 存储到字典中
    parse_datas = {
        "thoughts": thoughts_content,
        "sql": sql_data,
        "db_data": data_data,
        "display_type": type_data,
    }
    return parse_datas

import sqlite3


def execute_query(db_file, sql_query):
    """
    执行 SQL 查询
    """
    try:
        # 连接到 SQLite 数据库
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # 执行 SQL 查询
        cursor.execute(sql_query)

        # 获取字段名
        columns = [description[0] for description in cursor.description]

        # 获取查询结果
        rows = cursor.fetchall()

        # 将字段名和对应的值打包成字典
        result = [dict(zip(columns, row)) for row in rows]

        return result

    except sqlite3.Error as e:
        print(f"SQLite 错误: {e}")
        return f"SQLite 错误: {e}"

    finally:
        # 关闭数据库连接
        if conn:
            conn.close()

def main_test(url, model_name, user_input, db_file):
    """
    测试主函数
    """
    response = send_post_request(url, model_name, user_input, db_file)
    parse_res = parse_data_model_msg(response)
    sql = parse_res["sql"]
    print(f"sql:{sql}")
    # 查询结果并输出对应的字段名
    results = execute_query(db_file, sql)
    print(results)


In [17]:
# user_input = "2022年第四季度,报价有变动的零件数量占比是多少"


# main_test(
#     url=URL,
#     model_name=MODEL_NAME,
#     user_input=user_input,
#     db_file=DB_FILE
# )

## 评估数据处理
- 从excel、csv、数据库中读取query数据，进行去重等处理

In [18]:
# import pandas as pd

# # 读取excel文件，query去重等
# excel_path = "./BBT-M-27 68 AI Question List.xlsx"
# df_1 = pd.read_excel(excel_path, sheet_name="Real Question from Business")
# df_2 = pd.read_excel(excel_path, sheet_name="Real Question from Business 68")
# queries = df_1["Question"].drop_duplicates().dropna().tolist() + df_2["Question"].drop_duplicates().dropna().tolist()
# print(len(queries))


# 从txt文件中读取query
with open("query.txt", "r") as f:
    queries = f.readlines()
    queries = [query.strip() for query in queries]
print(len(queries))


54


## 推理结果处理

- 指定模型名称，模型名称与`.env`文件中的`MODEL_NAME`对应
- 结果存储到excel文件中，表头为`query`、`sql`、`result`、`log`，分别表示查询语句、推理的sql语句、推理结果、结果判断日志（Success、None、Error）
- 

In [19]:
import time
import pandas as pd
from tqdm import tqdm
from openpyxl import load_workbook


def evaluate_query(url: str, model_name: str, query_list: list, db_file: str, sleep_time: int = 0.5):
    """
    请求数据模型，执行 SQL 查询，并将结果写入 Excel 文件
    url: 数据模型的 API 地址
    model_name: 数据模型的名称
    query_list: 查询列表
    db_file: SQLite 数据库文件
    sleep_time: 请求数据模型的间隔时间
    """
    # 初始化Excel文件路径
    excel_file = f"{model_name}.xlsx"
    # 创建空的 DataFrame，确保表头只写入一次
    df_headers = pd.DataFrame(columns=["query", "sql", "result", "log"])
    df_headers.to_excel(excel_file, index=False, sheet_name=model_name)

    # 使用 openpyxl 打开工作簿
    book = load_workbook(excel_file)
    sheet = book.active


    # 
    success_count = 0
    error_count = 0
    none_count = 0
    for query in tqdm(query_list[:2], desc="Processing queries"):
        time.sleep(sleep_time) # 间隔n秒，防止请求过快
        print(query)
        response = send_post_request(url, model_name, query, db_file)
        parse_res = parse_data_model_msg(response)
        
        try:
            if parse_res.get("sql"):
                query_res = execute_query(db_file, parse_res["sql"])
            else:
                query_res = []
        except:
            query_res = []
        
        print(query_res)
        
        if "SQLite 错误" in str(query_res):
            log = "Error"
            error_count += 1
        elif len(query_res) == 0:
            log = "None"
            none_count += 1
        else:
            log = "Success"
            success_count += 1
        
        # 将当前查询结果添加到工作表中
        data = {"query": query, "sql": parse_res.get("sql", ""), "result": str(query_res), "log": log}
        sheet.append(list(data.values()))
        
        # 每处理10个查询就保存一次，以防程序意外终止
        if sheet.max_row % 10 == 0:
            book.save(excel_file)

        print("-"*50)
    # 最后保存工作簿
    book.save(excel_file)

    print(f"Success: {success_count}, Error: {error_count}, None: {none_count}")
    # 计算占比
    total = success_count + error_count + none_count
    print(f"Success Rate: {success_count/total:.2f}, Error Rate: {error_count/total:.2f}, None Rate: {none_count/total:.2f}")



## 开始评估

In [20]:
evaluate_query(
    url=URL,
    model_name=MODEL_NAME,
    query_list=queries,
    db_file=DB_FILE,
    sleep_time=1
)

Processing queries:   0%|          | 0/2 [00:00<?, ?it/s]

供应商A 都供哪些项目？


Processing queries:  50%|█████     | 1/2 [00:15<00:15, 15.96s/it]

[{'bmw_project': 'Project X'}]
--------------------------------------------------
零件5B3A058-01的报价总价是多少？


Processing queries: 100%|██████████| 2/2 [00:33<00:00, 16.64s/it]

[{'total_costs_9_0_X': 1750}]
--------------------------------------------------
Success: 2, Error: 0, None: 0
Success Rate: 1.00, Error Rate: 0.00, None Rate: 0.00
